# Data Preparation

### Preprocessing Data
**Koneksi Data ke Power BI** \
saya menggunakan Power BI untuk preprocessing data dari dataset iris. Meskipun Power BI tidak memiliki fungsi native untuk algoritma seperti KNN (K-Nearest Neighbors) atau ABOD (Angle-Based Outlier Detection), kita bisa menggunakan metode statistik yang powerful atau mengintegrasikan skrip Python.







1. Konfigurasi environment di Power BI \
Karena menggunakan Python Script untuk melakukan pemrosesan data dalam Power BI maka perlu kita konfigurasikan:
- Pertama, buka Power BI desktop
- Kemudian pergi ke menu `File > Options and Settings > Options`
- Setelah itu pilih `Python Scripting`
- Kemudian pilih _home directories_ yang memiliki library `pandas`, `pycaret`, `scikit-learn`, `matplotlib`, dan `seaborn`
- Setelah selesai kemudian klik OK



2. Implementasi pada Power Query di Power BI
A. Memuat data
Untuk memuat data dan menggabungkan data, pada menu `Home` klik `Transform Data` kemudian akan diarahkan ke Power Query. Setelah itu klik `Merge Queries`. Kemudian pilih tabel yang akan digabung, kemudian pilih Join Kind sesuai yang dibutuhkan (misalnya Left Join) kemudian klik OK. Setelah itu klik `Close & Apply`. 
B. PreProcessing data menggunakan Python Script
Untuk melakukan preprocessing data di Power BI, klik `Transform data` di menu Home. Setelah masuk ke Power Query, pilih `Run Python Script` kemudian tulis kode Python untuk melakukan preprocessing data. 



3. Mendeteksi Outlier menggunakan K-Nearest Neighbor (KNN) \
Penggunaan KNN untuk mendeteksi outlier adalah apabila sebuah titik data dianggap outlier jika ia terisolasi dari lingkungannya. Sebaliknya, titik data normal (inlier) akan memiliki banyak "tetangga" di dekatnya. 

Cara Kerja:
- Untuk setiap titik data, algoritma akan mencari K tetangga terdekatnya (misalnya, 5 tetangga terdekat).
- Kemudian, dihitung rata-rata jarak dari titik tersebut ke semua K tetangganya.
- Titik data yang memiliki rata-rata jarak paling besar dianggap sebagai outlier. Secara intuitif, jika Anda harus berjalan sangat jauh untuk menemukan tetangga terdekat Anda, kemungkinan besar Anda berada di lokasi yang terpencil (sebuah outlier).

Untuk analoginya adalah sebuah pesta. Orang-orang normal akan berkumpul dalam kelompok-kelompok kecil (inlier). Seseorang yang berdiri sendirian di sudut ruangan, jauh dari semua kelompok, adalah seorang outlier. KNN pada dasarnya mengukur "seberapa jauh seseorang dari kerumunan terdekatnya."

Untuk contoh dari script python sebagai berikut

In [ ]:
from pycaret.anomaly import *

# Inisialisasi lingkungan, abaikan kolom non-fitur
anomaly = setup(
    data=dataset,
    ignore_features=['id', 'species'],
    session_id=123,     # untuk reprodusibilitas
    verbose=False       # matikan pesan interaktif
)

# Latih model KNN
knn = create_model('knn')

predictions = assign_model(knn)
outliers = predictions[predictions['Anomaly'] == 0]

# Hapus outliers
outliers_removed = predictions[predictions['Anomaly'] == 0]


4. Analisis menggunakan ABOD (Angle-Based Outlier Detection) \
Metode ini yaitu apabila sebuah titik data dianggap outlier jika hubungan sudut antara dirinya dengan titik-titik data lain sangat bervariasi. Titik data normal (inlier) yang berada di dalam sebuah cluster akan memiliki sudut pandang yang lebih stabil dan sempit terhadap titik-titik lain.

Cara Kerja:
- Untuk setiap titik data (sebut saja titik P), algoritma akan mengambil pasangan titik lain (misalnya, A dan B).
- Kemudian, dihitung sudut yang terbentuk oleh vektor PA dan PB.
- Proses ini diulang untuk banyak pasangan titik lainnya.
- Jika variansi (keragaman) sudut yang terbentuk sangat tinggi, maka titik P kemungkinan besar adalah outlier karena posisinya berada di luar cluster yang padat. Sebaliknya, jika titik P berada di tengah cluster, sudut yang terbentuk akan cenderung lebih kecil dan seragam.

Untuk analoginya seperti Anda berdiri di tengah-tengah pasar yang ramai (sebagai inlier). Ke mana pun Anda melihat, pandangan Anda akan relatif sempit karena terhalang oleh orang-orang di sekitar Anda. Sekarang, bayangkan Anda berdiri di puncak bukit di pinggir pasar (sebagai outlier). Anda bisa melihat ke segala arah dengan sudut pandang yang sangat lebar dan bervariasi. ABOD mengukur "keragaman sudut pandang" ini.



Berikut contoh script dari ABOD

In [ ]:
from pycaret.anomaly import *

# Inisialisasi lingkungan
anomaly = setup(
    data=dataset,
    ignore_features=['id', 'species'],
    session_id=123,     # untuk reprodusibilitas
    verbose=False       # matikan pesan interaktif
)

# Latih model ABOD dengan asumsi 1% data adalah outlier
abod = create_model('abod', fraction=0.01)

predictions = assign_model(abod) 

#Hapus outliers
outliers_removed = predictions[predictions['Anomaly'] == 0]

5. Hasil Scripting \
Setelah menjalankan salah satu script maka dihasilkan data hasil yang mana data yang dianggap outlier dihilangkan, sehingga isi dari dataset sebagai berikut

In [1]:
# 1. Impor pustaka pandas
import pandas as pd

# 2. Tentukan nama file CSV Anda
file_knn = 'knn-iris.csv'

# 3. Baca file CSV ke dalam DataFrame baru
df_knn = pd.read_csv(file_knn)

print("Data Bersih Hasil KNN:")
print(df_knn.to_string())

Data Bersih Hasil KNN:
         Value.Class  Value.petal length  Value.petal width  Value.sepal length  Value.sepal width  Value.Anomaly  Value.Anomaly_Score
0        Iris-setosa                 1.0                0.2                 4.6                3.6              0             0.565685
1        Iris-setosa                 1.1                0.1                 4.3                3.0              0             0.500000
2        Iris-setosa                 1.2                0.2                 5.0                3.2              0             0.346410
3        Iris-setosa                 1.2                0.2                 5.8                4.0              0             0.583095
4        Iris-setosa                 1.3                0.2                 4.4                3.0              0             0.300000
5        Iris-setosa                 1.3                0.2                 4.4                3.2              0             0.316228
6        Iris-setosa            

In [2]:
# 1. Impor pustaka pandas
import pandas as pd

# 2. Tentukan nama file CSV Anda
file_abod = 'abod-iris.csv'

# 3. Baca file CSV ke dalam DataFrame baru
df_knn = pd.read_csv(file_abod)

print("Data Bersih Hasil ABOD:")
print(df_knn.to_string())

Data Bersih Hasil ABOD:
               Class  petal length  petal width  sepal length  sepal width  Anomaly          Anomaly_Score
0        Iris-setosa           1.0          0.2           4.6          3.6        0   '-0.7717452904177516
1        Iris-setosa           1.1          0.1           4.3          3.0        0    '-6.211703408463766
2        Iris-setosa           1.2          0.2           5.0          3.2        0   '-22.260507331818097
3        Iris-setosa           1.2          0.2           5.8          4.0        0   '-0.9712668356408708
4        Iris-setosa           1.3          0.2           4.4          3.0        0    '-128.1522455647601
5        Iris-setosa           1.3          0.2           4.4          3.2        0   '-44.802538833080554
6        Iris-setosa           1.3          0.2           4.7          3.2        0     '-93.4220000264197
7        Iris-setosa           1.3          0.2           5.5          3.5        0   '-3.6381891263247517
8        Iris